In [8]:
pip install tqdm pandas spacy gradio numpy fdf

^C
     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     -------------------------------------- 0.0/587.7 MB 660.6 kB/s eta 0:14:50
     ---------------------------------------- 0.1/587.7 MB 1.2 MB/s eta 0:08:11
     ---------------------------------------- 0.4/587.7 MB 2.5 MB/s eta 0:03:58
     ---------------------------------------- 0.9/587.7 MB 4.7 MB/s eta 0:02:06
     ---------------------------------------- 1.2/587.7 MB 5.1 MB/s eta 0:01:57
     ---------------------------------------- 1.8/587.7 MB 6.7 MB/s eta 0:01:28
     ---------------------------------------- 2.2/587.7 MB 7.0 MB/s eta 0:01:25
     ---------------------------------------- 2.9/587.7 MB 8.0 MB/s eta 0:01:13
     ---------------------------------------- 3.7/587.7 MB 8.7 MB/s eta 0:01:08
     ---------------------------------------- 3.9/587.7 MB 8.8 MB/s eta 0:01:07
     ---------------------------------------- 4.4/587.7 MB 8.8 MB/s eta 0:01:07
     ---------------------------------------

In [8]:
# put this in the terminal and run: python -m spacy download en
# Python env == 3.11.9

^C
     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     -------------------------------------- 0.0/587.7 MB 660.6 kB/s eta 0:14:50
     ---------------------------------------- 0.1/587.7 MB 1.2 MB/s eta 0:08:11
     ---------------------------------------- 0.4/587.7 MB 2.5 MB/s eta 0:03:58
     ---------------------------------------- 0.9/587.7 MB 4.7 MB/s eta 0:02:06
     ---------------------------------------- 1.2/587.7 MB 5.1 MB/s eta 0:01:57
     ---------------------------------------- 1.8/587.7 MB 6.7 MB/s eta 0:01:28
     ---------------------------------------- 2.2/587.7 MB 7.0 MB/s eta 0:01:25
     ---------------------------------------- 2.9/587.7 MB 8.0 MB/s eta 0:01:13
     ---------------------------------------- 3.7/587.7 MB 8.7 MB/s eta 0:01:08
     ---------------------------------------- 3.9/587.7 MB 8.8 MB/s eta 0:01:07
     ---------------------------------------- 4.4/587.7 MB 8.8 MB/s eta 0:01:07
     ---------------------------------------

In [1]:
from decimal import Decimal
import pandas as pd
from tqdm.auto import tqdm
import math
import spacy
import re
from collections import Counter
import requests
import random as rd
import string
import gradio as gr
import math
import numpy as np
from fpdf import FPDF

# Load 界定 Lexcial Sophisitication 詞彙的 corpus
bnc = pd.read_excel("BNC.xlsx")
not_nslex = list(bnc["Word"][:2000]) # 前兩千個不是複雜字彙
not_vslex = list(bnc["POS-tag"][:2000])

In [2]:

nlp = spacy.load("en_core_web_lg") 

def pos(attempt):
    pos_translation = []
    meaning_pos = ["VERB", "ADJ", "ADV", "NOUN",
                   "DET", "ADP", "CONJ", "NUM", "PRT", "PRON", "PROPN"]
    
    doc = nlp(attempt)
    
    pos_sent = []
    word_list = []
    for token in doc:
        word_list.append(token.text)
        pos_sent.append(token.pos_)

    return pos_sent, word_list


In [3]:
level_val = {'A1': 0, 'A2': 1,"B1":2,"B2":3,"C1":4} # 用來計算及分類等級的 Dict

# 用來找使用頻率大於等於2的詞彙
def find_frequent_words(text, min_count=2):
    words = re.findall(r'\b\w+\b', text.lower())
    frequency = Counter(words)

    frequent_words = {word: count for word, count in frequency.items() if count >= min_count}
    
    return frequent_words

#用來找尋替代的字彙，亦即建議學生可以使用那些較複雜的字彙 (最相關的)
def suggest_sophlex(text):
    bnc = pd.read_excel("BNC.xlsx")
    not_nslex = list(bnc["Word"][:2000])
    not_sophlex = ls_cal(text)[2]
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',}
    if len(not_sophlex)>10:
        new_not_sophlex = rd.sample(not_sophlex,11)
    else:
        new_not_sophlex = not_sophlex
    
    suggestion={}
    
    for t in new_not_sophlex:
        r = requests.get(f"https://www.thesaurus.com/browse/{t}", headers=headers)
        if re.search("Strongest matches", r.text) is not None: 
            synonym_page = r.text.split("Strongest matches")[1].split("Strong matches")[0]
            synonyms = re.findall(r"http:\/\/www\.thesaurus\.com\:80\/browse\/(\w+)\"\sclass=",synonym_page)
            soph_syns = [s for s in synonyms if s not in not_nslex]
            if soph_syns !=[]:
                suggestion[t]=soph_syns
            
    return suggestion
    
#用來找尋替代的字彙，亦即建議學生可以使用那些較複雜的字彙 (次相關的)
def suggestions(text):
    repetition = LexVar_calculation(text)[2]
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',}
    suggestion={}
    
    for t in repetition:
        r = requests.get(f"https://www.thesaurus.com/browse/{t}", headers=headers)
        if re.search("Strong matches", r.text) is not None: 
            synonym_page = r.text.split("Strong matches")[1].split("Strong matches")[0]
            synonyms = re.findall(r"http:\/\/www\.thesaurus\.com\:80\/browse\/(\w+)\"\sclass=",synonym_page)
            soph_syns = [s for s in synonyms if s not in not_nslex]
            if soph_syns !=[]:
                suggestion[t]=soph_syns
            
    return suggestion

#計算各種詞彙類型的用字比率
def find_ratio(text):
    word = text.translate(str.maketrans('', '', string.punctuation))
    words = re.findall(r'\b\w+\b',word.lower())
    pos_text = pos(word)
    function_words = []
    verb = []
    noun = []
    adj = []
    adv = []
    ratio = {}
    for index,text in enumerate(pos_text[0]):
        if text == "ADP" or text =="CCONJ" or text=="PRON" or text =="DET" or text=="AUX" or text =="SCONJ":
            function_words.append(pos_text[1][index])
        elif text == "VERB":
            verb.append(pos_text[1][index])
        elif text == "NOUN":
            noun.append(pos_text[1][index])
        elif text == "ADJ":
            adj.append(pos_text[1][index])
        elif text == "ADV":
            adv.append(pos_text[1][index])
            
    ratio[f"Function words, examples: {','.join(map(str,function_words[:3]))}"] = int(round(len(function_words)/len(words)*100,0))
    ratio[f"Verbs, examples: {','.join(map(str,verb[:3]))}"] = int(round(len(verb) / len(words)*100,0))
    ratio[f"Nouns, examples: {','.join(map(str,noun[:3]))}"] = int(round(len(noun) / len(words)*100,0))
    ratio[f"Adjetives, examples: {','.join(map(str,adj[:3]))}"] = int(round(len(adj) / len(words)*100,0))
    ratio[f"Adverbs, examples: {','.join(map(str,adv[:3]))}"] = int(round(len(adv) / len(words)*100,0))

    return ratio

#用來計算 lexical Density Ratio
def LexDen_calculation(text):
    LD = []
    content_words = []
    pos_text = pos(text)
    for s in pos_text[0]:
        content_word = []
        if s == "VERB" or s == "NOUN" or s == "ADJ" or s == "ADV":
            content_word.append(1)
        content_words.append(sum(content_word))
        LD.append(float(Decimal(sum(content_word))/Decimal(len(pos_text[0]))))           

    ld = round(sum(LD),2)

    #以下這個分數及具有跑過統計
    if ld<=0.31:ld_eval = "A1"
    elif ld>0.31 and ld<=0.38: ld_eval ="A2"
    elif ld>0.38 and ld<=0.39: ld_eval = "B1"
    elif ld>0.39 and ld<0.4: ld_eval = "B2"
    elif ld>=0.41:ld_eval = "C1"
        
    result = level_val.get(ld_eval)
    if result == 0: lexden_lev = "A1"
    elif result == 1: lexden_lev ="A2"
    elif result == 2: lexden_lev = "B1"
    elif result == 3: lexden_lev = "B2"
    elif result == 4: lexden_lev = "C1"
    
    
    ratio = find_ratio(text)
    
    return ld,ld_eval,ratio,lexden_lev

#用來計算 lexical Sophistication Ratio    
def ls_cal(t):
    puncts = [",", ".", "-", "/", "?", "!", ":", ";", "–", "{", "}", "[", "]", "(", ")", "'", '"', "*"]
    bnc = pd.read_excel("BNC.xlsx")
    not_nslex = list(bnc["Word"][:2000])
    not_vslex = list(bnc["POS-tag"][:2000])
    tokens = nlp(t.lower())
    sent_pos = pos(t)

    content_ws = []
    for j, pos_tag in enumerate(sent_pos[0]):
        if pos_tag == "VERB" or pos_tag == "NOUN" or pos_tag == "ADJ" or pos_tag == "ADV" or pos_tag=="PROPN":
            content_ws.append(tokens[j].lemma_)


    tokens = [w.lemma_ for w in tokens if w.lemma_ != "" and w.lemma_ != " " and w.lemma_ not in puncts]
    sophlex = [t for t in content_ws if t not in not_nslex]
    not_sophlex = [t for t in content_ws if t in not_nslex]
    LS1 = round((float(Decimal(len(sophlex))/Decimal(len(content_ws)))),2)
    
    if LS1 <=0.45: ls1_eval = "A1"
    elif LS1 >0.45 and LS1<=0.87: ls1_eval = "A2"
    elif LS1 >0.87 and LS1<=0.99: ls1_eval = "B1"
    elif LS1 >0.99 and LS1<=1.29: ls1_eval = "B2"
    elif LS1 >1.29: ls1_eval = "C1"

    result = level_val.get(ls1_eval)
    
    if result == 0: ls1_lev = "A1"
    elif result == 1: ls1_lev ="A2"
    elif result == 2: ls1_lev = "B1"
    elif result == 3: ls1_lev = "B2"
    elif result == 4: ls1_lev = "C1"    
        
    return LS1,ls1_eval, not_sophlex, ls1_lev
    
#用來計算 lexical Variety Ratio
def LexVar_calculation(text):
    puncts = [",", ".", "-", "/", "?", "!", ":", ";", "–", "{", "}", "[", "]", "(", ")", "'", '"', "*"]
    tokens = " ".join(text.split("***")).replace(" ---", "")
    tokens = nlp(tokens)
    tokens = [w.lemma_ for w in tokens if w.lemma_ != "" and w.lemma_ != " " and w.lemma_ not in puncts]
    types = list(set(tokens))
    frequency = Counter(tokens)
    frequency.pop("the",None)
    frequency.pop("be",None)
    frequency.pop("I",None)
    frequency.pop("in",None)
    frequency.pop("on",None)
    frequency.pop("to",None)
    frequency.pop("by",None)
    frequency.pop("at",None)
    frequency.pop("a",None)
    frequency.pop("an",None)
    frequency.pop("an",None)
    frequency.pop("and",None)
    frequency.pop("you",None)
    frequency.pop("it",None)
    frequency.pop("for",None)

    
    min_count = int(len(tokens)*0.03) # 假定權重為 0.03，如果超過這個次數的字就被視為常出現的詞彙
    frequent_words = {word: count for word, count in frequency.items() if count > min_count}
    
    cttr = round(float(Decimal(len(types))/Decimal(math.sqrt(int(2*Decimal(len(tokens)))))) ,2)
    
    if cttr <=3.35: cttr_eval = "A1"
    elif cttr >3.35 and cttr<=3.92: cttr_eval = "A2"
    elif cttr >3.92 and cttr<=4.55: cttr_eval = "B1"
    elif cttr >4.55 and cttr<=4.98: cttr_eval = "B2"
    elif cttr >4.98: cttr_eval = "C1"


    result = level_val.get(cttr_eval)
    
    if result == 0: lexvar_lev = "A1"
    elif result == 1: lexvar_lev ="A2"
    elif result == 2: lexvar_lev = "B1"
    elif result == 3: lexvar_lev = "B2"
    elif result == 4: lexvar_lev = "C1"

    return cttr, cttr_eval,frequent_words,lexvar_lev,min_count

#根據用戶選擇的 level 與實際的等級做比對，並給予相應的回饋
def feedback_provision(exp_level, target_eval, index_name):
    level_conversion = [level_val.get(item) for item in [exp_level, target_eval]]
    exp_, target_ = level_conversion[0], level_conversion[1]

    if index_name == "Lexical Density":
        if exp_ < target_: feedback = "Great! You have exceeded the level you chose.!"
        elif exp_ > target_: feedback = "You may use more content words."
        if exp_ == target_: feedback = "Good! You have reached the level you chose.!"
    elif index_name == "Lexical Varation":
        if exp_ < target_: feedback = "Great! You have exceeded the level you chose.!"
        elif exp_ > target_: feedback = "You may use more different types of words.."
        if exp_ == target_: feedback = "Good! You have reached the level you chose.!"
    elif index_name == "Lexical Sophistication":
        if exp_ < target_: feedback = "Great! You have exceeded the level you chose.!"
        elif exp_ > target_: feedback = "You may use more sophisticated words."
        if exp_ == target_: feedback = "Good! You have reached the level you chose.!"

    return feedback


#用來製作PDF個人化報告的格式
class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Evaluation Feedback', 0, 1, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 15, title, 0, 1, 'L')
#         self.ln(10)

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, body)
        self.ln()

    def create_table(self, col_widths, data, header=None):
        self.set_font('Arial', 'B', 12)
        if header:
            for col_width, header_text in zip(col_widths, header):
                self.cell(col_width, 10, header_text, 1, 0, 'C')
            self.ln()
        
        self.set_font('Arial', '', 12)
        for row in data:
            for col_width, cell in zip(col_widths, row):
                self.cell(col_width, 10, str(cell), 1)
            self.ln()
            
#PDF個人化報告內容
def pdf_feedback(holistic_lev, text, exp_level, lv_eval, ld_eval, ls_eval):
    pdf = PDF()
    pdf.add_page()
    words = re.findall(r'\b\w+\b',text.lower())
    
    pdf.set_font('Arial', 'b', 12)
    pdf.cell(0, 15, 'Holistic Level: '+holistic_lev, 0, 1)

    cttr, cttr_eval, frequent_words, lexvar_lev, threshold= LexVar_calculation(text)
    ld, ld_eval,ratios,lexden_lev = LexDen_calculation(text)
    ls, ls_eval, not_sophlex, ls_lev = ls_cal(text)
    suggestion = suggest_sophlex(text)
    suggestion_var = suggestions(text)
    
    pdf.cell(0, 15, 'Lexical Variation Level: '+cttr_eval, 0, 1)
    pdf.cell(0, 15, 'Lexical Density Level: '+ld_eval, 0, 1)
    pdf.cell(0, 15, 'Lexical Sophistication Level: '+ls_eval, 0, 1)

    pdf.set_font('Arial', '', 12)

    level_conversion = [level_val.get(item) for item in [exp_level, lv_eval, ld_eval, ls_eval]]
    exp_, target_ = level_conversion[0], level_conversion[1]
    if exp_ > level_val.get(lv_eval):
        pdf.set_font('Arial', 'B', 12)
        pdf.multi_cell(0,10,f'Lexical Variation Feedback: You did not reach the level you chose. Words that appear more than {threshold} times are listed in the table below. Refer to the suggested replacements of the words also.')
        pdf.chapter_title('                         Lexical Density Feedback: Function words that appear more than twice:')

        if frequent_words:
            pdf.create_table([80, 110], [[word, f"{count} times"] for word, count in frequent_words.items()], ['Word', 'Count'])
            pdf.create_table([80, 110], [[word, ', '.join(sophlex[:3])] for word, sophlex in suggestion_var.items()], ['Your Word', 'Suggested Replacements'])

        else:
            pdf.chapter_body(f'No words appeared more than {threshold} times. This means you have to increase the length of your writing.')
    else:
        pdf.chapter_title('Lexical Variation Feedback: Good! You reached the level you chose.')

            
    if exp_ > level_val.get(ld_eval):
        pdf.set_font('Arial', 'B', 12)
        pdf.multi_cell(0,10,'Lexical Density Feedback: You did not reach the level you choose. Please refer to the following percentage of function words and content words, try increasing the length of your writing and reducing the use of function words.')        
        pdf.chapter_title('                         Lexical Density Feedback: Function words that appear more than twice:')

        pdf.create_table([110, 80], [[word, f"{ratio} %"] for word, ratio in ratios.items()], ['Word Types', 'Percentage'])
    
    else:
        pdf.chapter_title('Lexical Density Feedback: Good! You reached the level you chose.')

    
    if exp_ > level_val.get(ls_eval):
        pdf.set_font('Arial', 'B', 12)
        pdf.multi_cell(0,10,'Lexical Sophistication Feedback: You did not reach the level you chose. Please refer to the following suggested replacements and make sure the use of the words before applying them to your writing')        
        pdf.chapter_title('                         Lexical Density Feedback: Function words that appear more than twice:')

        if suggestion:
            pdf.create_table([80, 110], [[word, ', '.join(sophlex[:3])] for word, sophlex in suggestion.items()], ['Your Word', 'Suggested Replacements'])
        else:
            pdf.chapter_body('No suggestions for sophisticated words.')
    else:
        pdf.chapter_title('Lexical Sophisitication Feedback: Good! You reached the level you chose.')


    file_path = 'evaluation_feedback.pdf'
    pdf.output(file_path)
    return file_path

# 用來將前面的函式進行整合，因為 gradio只接受函式傳遞
def calculate_metrics(text, exp_level):
    cttr, cttr_eval, frequent_words, lexvar_lev,threshold = LexVar_calculation(text)
    ld, ld_eval,function_words,lexden_lev = LexDen_calculation(text)
    ls,ls_eval, not_sophlex, ls1_lev = ls_cal(text)
    sophlex = suggest_sophlex(text)
    result = np.array([level_val.get(item) for item in [lexvar_lev, lexden_lev, ls1_lev]])
    avg_result = round(result.mean(),0)

    if avg_result == 0: holistic_lev = "A1"
    elif avg_result == 1: holistic_lev = "A2"
    elif avg_result == 2: holistic_lev = "B1"
    elif avg_result == 3: holistic_lev = "B2"
    elif avg_result == 4: holistic_lev = "C1"

    cttr_feedback = feedback_provision(exp_level, cttr_eval, "Lexical Varation")
    ld_feedback = feedback_provision(exp_level, ld_eval, "Lexical Density")
    ls_feedback = feedback_provision(exp_level, ls_eval, "Lexical Sophistication")

    feedback = pdf_feedback(holistic_lev,text,exp_level,cttr_eval,ld_eval,ls_eval)
    

    holistic_lev_def = "整體的寫作等級是根據Lexical Variation、Lexical Density及Lexical Sophisitication所平均計算出來的結果。"
    lv_def = "衡量一段文本中詞彙多樣性程度的指標。高詞彙多樣性表示使用了很多不同的詞，而低詞彙多樣性則表示重複使用了相同的詞。"
    ld_def = "衡量一段文本中詞彙密集度的指標。高詞彙密集度表示使用了很多實詞（如名詞、動詞、形容詞、副詞）。低詞彙密集度表示使用了較少的實詞。"
    ls_def = "衡量一段文本中詞彙複雜度的指標。高詞彙複雜度表示使用了難度較高的單字。低詞彙複雜度表示使用了較簡單的單字。"
    
    lv_range =[["A1","<= 3.35"],["A2","3.36 ~ 3.92"],["B1","3.93 ~ 4.55"],["B2","4.56 ~ 4.98"],["C1",">= 4.99"]]
    ld_range =[["A1","<= 0.31"],["A2","0.32 ~ 0.37"],["B1","0.38"],["B2","0.39 ~ 0.4"],["C1",">= 0.41"]]
    ls_range =[["A1","<= 0.45"],["A2","0.46 ~ 0.87"],["B1","0.88 ~ 0.99"],["B2","1.0 ~ 1.29"],["C1",">= 1.3"]]

  
    return holistic_lev_def, holistic_lev, feedback, lv_def, cttr_eval, cttr,lv_range, cttr_feedback, ld_def, ld_eval, ld, ld_range, ld_feedback,ls_def, ls_eval, ls,ls_range, ls_feedback

#實際介面設計
with gr.Blocks() as demo:
    gr.Markdown("### Writing Evaluation and Assitance System")
    exp_level = gr.Dropdown(["A1", "A2","B1","B2","C1"], label="Expected Level")
    text_input = gr.Textbox(label="Please paste your writing here")
    sub_btn = gr.Button("Submit",variant="primary")

    with gr.Accordion("Evaluation Feedback"):
        holistic_def = gr.Text(label="Holistical Level Definition")
        holistic_eval_level_output = gr.Text(label="Holistic Level")
        pdf_output = gr.File(label="Download Evaluation Report")
    
    
    with gr.Accordion("Analytics"):

        with gr.Tab("Lexical Variation"):
            lv_def = gr.Text(label="Definition")
            cttr_eval_level_output = gr.Text(label="Lexical Variation Level")
            output_cttr = gr.Number(label="Lexical Variation Value")
            lv_range = gr.Dataframe(label="Value Range",
                                    headers=["CEFR Level","Value"],datatype=["str","str"],
                                    row_count=2,col_count=2)
            cttr_eval_feedback_output = gr.Text(label="Lexical Variation Feedback")
            

        with gr.Tab("Lexical Density"):
            ld_def = gr.Text(label="Definition")
            ld_level_output = gr.Text(label="Lexical Density Level")
            output_ld = gr.Number(label="Lexical Density Value")
            ld_range = gr.Dataframe(label="Value Range",
                                    headers=["CEFR Level","Value"],datatype=["str","str"],
                                    row_count=2,col_count=2)
            ld_eval_feedback_output = gr.Text(label="Lexical Density Feedback")
            
        with gr.Tab("Lexical Sophistication"):
            ls_def = gr.Text(label="Definition")
            ls_eval_level_output = gr.Text(label="Lexical Sophistication Level")
            output_ls = gr.Number(label="Lexical Sophistication Value")
            ls_range = gr.Dataframe(label="Value Range",
                                    headers=["CEFR Level","Value"],datatype=["str","str"],
                                    row_count=2,col_count=2)
            ls_eval_feedback_output = gr.Text(label="Lexical Sophistication Feedback")


    sub_btn.click(fn=calculate_metrics,
            inputs=[text_input, exp_level],
            outputs=[holistic_def,holistic_eval_level_output,pdf_output, lv_def,
                 cttr_eval_level_output, output_cttr, lv_range, cttr_eval_feedback_output,
                 ld_def,ld_level_output, output_ld, ld_range, ld_eval_feedback_output,
                 ls_def,ls_eval_level_output, output_ls,ls_range, ls_eval_feedback_output])

demo.launch(debug=False)


C:\Users\hhfeng\anaconda3\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.41.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
